In [1]:
!pip install pytorch_lightning

     |████████████████████████████████| 6.0MB 3.4MB/s 
     |████████████████████████████████| 829kB 47.3MB/s 
  Created wheel for pytorch-lightning: filename=pytorch_lightning-0.7.1-cp36-none-any.whl size=145306 sha256=d146c585daa996aeb73b317440f5e8183b6bf204cfcbd5074cfbc3d764a1e59c
  Stored in directory: /root/.cache/pip/wheels/dc/93/61/14094d2116ff739513dda993007501ae5701b78386b39d5912
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=c09fbb1cb7aa2d8d8f47df1712b997b21145a30350bb05b27efaeb848a912545
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built pytorch-lightning future
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [2]:
!pip install comet_ml

     |████████████████████████████████| 194kB 3.4MB/s 
     |████████████████████████████████| 409kB 10.4MB/s 
     |████████████████████████████████| 204kB 18.8MB/s 
  Created wheel for configobj: filename=configobj-5.0.6-cp36-none-any.whl size=34546 sha256=766111299f8310e307e31d274493fdd4afeda58fd9d71f5fd53049bae34e62a3
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj


In [3]:
# used fo TPU
import collections
from datetime import datetime, timedelta
import os
import requests
import threading

_VersionConfig = collections.namedtuple('_VersionConfig', 'wheels,server')
VERSION = "xrt==1.15.0"  #@param ["xrt==1.15.0", "torch_xla==nightly"]
CONFIG = {
    'xrt==1.15.0': _VersionConfig('1.15', '1.15.0'),
    'torch_xla==nightly': _VersionConfig('nightly', 'XRT-dev{}'.format(
        (datetime.today() - timedelta(1)).strftime('%Y%m%d'))),
}[VERSION]
DIST_BUCKET = 'gs://tpu-pytorch/wheels'
TORCH_WHEEL = 'torch-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)
TORCH_XLA_WHEEL = 'torch_xla-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)
TORCHVISION_WHEEL = 'torchvision-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)

# Update TPU XRT version
def update_server_xrt():
  print('Updating server-side XRT to {} ...'.format(CONFIG.server))
  url = 'http://{TPU_ADDRESS}:8475/requestversion/{XRT_VERSION}'.format(
      TPU_ADDRESS=os.environ['COLAB_TPU_ADDR'].split(':')[0],
      XRT_VERSION=CONFIG.server,
  )
  print('Done updating server-side XRT: {}'.format(requests.post(url)))

update = threading.Thread(target=update_server_xrt)
update.start()

Updating server-side XRT to 1.15.0 ...


In [0]:
import os

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms, datasets
import torch.nn as nn

import torch.distributions.bernoulli

import pytorch_lightning as pl
from pytorch_lightning import loggers
from pytorch_lightning.callbacks import ModelCheckpoint

from PIL import Image

In [0]:
class invertColor():
    """
    Is used to inverse the color of a dataset when this dataset is in range[0;1]
    """
    def __call__(self, img):
        return self.invert(img)

    def invert(self, img):
        return 1 - img

In [0]:
class randomBlackToWhite():
    """
    Is used to transform some white pixels of the digit in black pixel
    """
    def __init__(self, prob):
      self.prob = prob

    def __call__(self, img):
        return self.randomBlackToWhite(img)

    def randomBlackToWhite(self, img, prob=0.9):
        return img * torch.distributions.bernoulli.Bernoulli(torch.tensor([self.prob])).sample(img.shape).squeeze(-1)

In [0]:
class CNN(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.batch_size = batch_size
        self.lr = lr
        self.ndf = ndf

        self.val_correct_counter = 0
        self.val_total_counter = 0

        self.hidden0 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=self.ndf, kernel_size=4),
            nn.LeakyReLU(0.2)
        )

        self.hidden1 = nn.Sequential(
            nn.Conv2d(self.ndf, self.ndf * 4, kernel_size=4),
            nn.LeakyReLU(0.2)
        )

        self.hidden2 = nn.Sequential(
            nn.Conv2d(self.ndf * 4, self.ndf, kernel_size=4),
            nn.LeakyReLU(0.2)
        )

        self.hidden3 = nn.Sequential(
            nn.Linear(5776, 1000),
            nn.LeakyReLU(0.2)
        )

        self.hidden4 = nn.Sequential(
            nn.Linear(1000, 200),
            nn.LeakyReLU(0.2)
        )

        self.hidden5 = nn.Sequential(
            nn.Linear(200, 10)
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)

        x = torch.flatten(x, start_dim=1, end_dim=-1)

        x = self.hidden3(x)
        x = self.hidden4(x)
        x = self.hidden5(x)

        return x

    def cross_entropy_loss(self, predicted_label, label):
        return F.cross_entropy(predicted_label, label)

    def training_step(self, batch, batch_idx):
        x, y = batch

        predicted = self.forward(x)
        loss = self.cross_entropy_loss(predicted, y)

        logs = {"train_loss": loss}
        return {"loss": loss, "log": logs}

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch

        predicted = self.forward(x)
        loss = self.cross_entropy_loss(predicted, y)

        comet_logger.experiment.log_confusion_matrix(labels=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"],
                                                     y_true=torch.eye(10)[y].view(-1, 10),
                                                     y_predicted=predicted
                                                     )

        self.val_correct_counter += int((torch.argmax(predicted, 1).flatten() == y).sum())
        self.val_total_counter += y.size(0)

        logs = {"val_loss": loss}
        return {"val_loss": loss, "log": logs}


    def validation_epoch_end(self, outputs):
        # outputs is an array with what you returned in validation_step for each batch
        # outputs = [{'loss': batch_0_loss}, {'loss': batch_1_loss}, ..., {'loss': batch_n_loss}]

        avg_acc = 100 * self.val_correct_counter / self.val_total_counter

        self.val_correct_counter = 0
        self.val_total_counter = 0

        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'avg_val_acc': avg_acc, 'val_loss': avg_loss}
        return {'avg_val_acc': avg_acc, 'avg_val_loss': avg_loss, 'log': tensorboard_logs}

    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        y_hat = self.forward(x)

        loss = self.cross_entropy_loss(y_hat, y)

        self.test_correct_counter += int((torch.argmax(y_hat, 1).flatten() == y).sum())
        self.test_total_counter += y.size(0)

        return {"test_loss": loss}

    def test_epoch_end(self, outputs):
        avg_acc = 100 * self.test_correct_counter / self.test_total_counter

        self.test_correct_counter = 0
        self.test_total_counter = 0

        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        tensorboard_logs = {'avg_test_acc': avg_acc, 'test_loss': avg_loss}

        return {"avg_test_loss": avg_loss, "avg_test_acc": avg_acc, "log: ": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def prepare_data(self):
        compose = transforms.Compose([
            transforms.RandomCrop(28, padding=8, pad_if_needed=True, fill=1, padding_mode='edge'),
            transforms.ToTensor(),
            randomBlackToWhite(0.7),
            invertColor(),
            transforms.Normalize((0.5,), (0.5,))
        ])

        self.mnist_train = datasets.MNIST(
            root="data",
            train=True,
            download=True,
            transform=compose
        )

        self.mnist_test = datasets.MNIST(
            root="data",
            train=False,
            download=True,
            transform=compose
        )

        self.mnist_train, self.mnist_val = torch.utils.data.random_split(self.mnist_train, [55000, 5000])

    def train_dataloader(self):
        mnist_train_loader = torch.utils.data.DataLoader(self.mnist_train,
                                                         batch_size=self.batch_size,
                                                         num_workers=1,
                                                         shuffle=True)

        return mnist_train_loader

    def val_dataloader(self):
        mnist_val_loader = torch.utils.data.DataLoader(self.mnist_val,
                                                         batch_size=self.batch_size,
                                                         num_workers=1,
                                                         shuffle=True)

        return mnist_val_loader

    def test_dataloader(self):
        mnist_test_loader = torch.utils.data.DataLoader(self.mnist_test,
                                                       batch_size=self.batch_size,
                                                       num_workers=1,
                                                       shuffle=True)

        return mnist_test_loader

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

In [0]:
# Parameters
experiment_name = "cnn_mnist_Bernoulli"
dataset_name = "MNIST_inv_color"
checkpoint_folder = "./" + experiment_name + "_checkpoints/"
tags = ["cnn", "real", "inverse_color", "Bernoulli", "randomly_padded"]
lr = 0.001
batch_size = 128*6
ndf = 16

In [9]:
# init logger
comet_logger = loggers.CometLogger(
    api_key="lcHaEOsylE8TDnW7qvtwek038",
    rest_api_key="P2y9z4euMGzYEEZHdR0j8I0xH",
    project_name="mnist-classifier",
    experiment_name=experiment_name,
)

#defining net
net = CNN()

#logging
comet_logger.experiment.set_model_graph(str(net))
comet_logger.experiment.add_tags(tags=tags)
comet_logger.experiment.log_dataset_info(name=dataset_name)
comet_logger.experiment.log_parameter(name="learning_rate", value=lr)
comet_logger.experiment.log_parameter(name="batch_size", value=batch_size)
comet_logger.experiment.log_parameter(name="ndf", value=ndf)

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/luposx/mnist-classifier/e9976885f2d84910a87696f225065a64



In [10]:
 # Training the NN
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_folder, save_top_k=3)
trainer = pl.Trainer(# fast_dev_run=True,
                      checkpoint_callback=checkpoint_callback,
                      max_epochs=40,
                      logger=comet_logger)
trainer.fit(net)

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!





Done updating server-side XRT: <Response [200]>


COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.ml/luposx/mnist-classifier/e9976885f2d84910a87696f225065a64
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     avg_val_acc [40]        : (70.44, 97.26)
COMET INFO:     sys.cpu.percent.01 [92] : (30.3, 95.2)
COMET INFO:     sys.cpu.percent.02 [92] : (8.1, 94.2)
COMET INFO:     sys.cpu.percent.avg [92]: (19.2, 64.1)
COMET INFO:     sys.ram.total [92]      : (13655232512.0, 13655232512.0)
COMET INFO:     sys.ram.used [92]       : (629481472.0, 3035832320.0)
COMET INFO:     train_loss [320]        : (0.05546760931611061, 2.342285633087158)
COMET INFO:     val_loss [40]           : (0.09795729070901871, 0.8859792351722717)
COMET INFO:   Other [count]:
COMET INFO:     Name        : cnn_mnist_Bernoulli
COMET INFO:     dataset_info: MNIST_inv_color
COMET INFO:   Uploads:
COMET INFO:     confusion-matrix: 285
COMET INFO: ----------------------------

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


1

In [11]:
# log checkpoints
comet_logger.experiment.log_asset_folder(folder=checkpoint_folder)

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/luposx/mnist-classifier/e9976885f2d84910a87696f225065a64



[('_ckpt_epoch_34.ckpt',
  {'api': 'https://www.comet.ml/api/rest/v2/experiment/asset/get-asset?assetId=c3c63932399b4ccca648af08f05335d7&experimentKey=e9976885f2d84910a87696f225065a64',
   'assetId': 'c3c63932399b4ccca648af08f05335d7',
   'web': 'https://www.comet.ml/api/asset/download?assetId=c3c63932399b4ccca648af08f05335d7&experimentKey=e9976885f2d84910a87696f225065a64'}),
 ('_ckpt_epoch_36.ckpt',
  {'api': 'https://www.comet.ml/api/rest/v2/experiment/asset/get-asset?assetId=2f0fc31be31d4b02afed7f737dc1b971&experimentKey=e9976885f2d84910a87696f225065a64',
   'assetId': '2f0fc31be31d4b02afed7f737dc1b971',
   'web': 'https://www.comet.ml/api/asset/download?assetId=2f0fc31be31d4b02afed7f737dc1b971&experimentKey=e9976885f2d84910a87696f225065a64'}),
 ('_ckpt_epoch_38.ckpt',
  {'api': 'https://www.comet.ml/api/rest/v2/experiment/asset/get-asset?assetId=5a05820f7b4b466896ba907e19d57e5a&experimentKey=e9976885f2d84910a87696f225065a64',
   'assetId': '5a05820f7b4b466896ba907e19d57e5a',
   'we

In [12]:
a

NameError: ignored